In [1]:
import os
import tqdm
import time
import datetime
import pandas as pd
import requests

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from xml.etree import ElementTree as ET
from xml.dom import minidom

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("start-maximized")

In [3]:
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)

In [4]:
RECIPES_INFOR_DIR = './data/recipes'
os.makedirs('./data', exist_ok=True)
os.makedirs(RECIPES_INFOR_DIR, exist_ok=True)

In [6]:
MAIN_URL = 'https://pinchofyum.com/recipes'

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)
driver.get(MAIN_URL)

In [3]:

MAIN_URL = 'https://pinchofyum.com/recipes'
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)
driver.get(MAIN_URL)

response = requests.get(MAIN_URL)
soup = BeautifulSoup(response.text, 'html.parser')

sections = soup.find_all('section', 'py-8 px-4 mx-auto md:max-w-5xl recipe-lists')
recipe_types: dict[str, dict] = {}
data = ET.Element('data')
for section in sections:
    recipe_type: str = section.find('h2').text
    recipe_type_children = section.find_all('a')
    recipe_types[recipe_type] = {}
    recipe_type_tag = ET.SubElement(data, 'parenttype', {'name': recipe_type})

    for recipe_type_child in recipe_type_children:
        recipe_types[recipe_type][recipe_type_child.text] = recipe_type_child['href']
        recipe_type_child_tag = ET.SubElement(recipe_type_tag, 'childtype', name=recipe_type_child.text)

def prettify_xml(elem):
    rough_string = ET.tostring(elem, encoding="unicode")
    parsed = minidom.parseString(rough_string)
    return parsed.toprettyxml(indent="\t")

FILE_PATH = './data/recipe_types/recipe_types.xml'

with open(FILE_PATH, 'w', encoding='utf-8') as f:
    f.write(prettify_xml(data))
recipe_types['Popular Categories'].pop('All Recipes')

'https://pinchofyum.com/recipes/all'

In [4]:
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)

# root cua file xml
data = ET.Element('data')

# -----------------------------------------------------------------
for recipe_type_name, child_types in tqdm.tqdm(recipe_types.items()):

    recipe = ET.Element('recipe')

    # Lay noi dung parent type recipe
    parenttype      = ET.SubElement(recipe, 'parenttype')
    parenttype.text = recipe_type_name

    recipe_type_tag = ET.SubElement(data, 'parenttype', {'name': recipe_type_name})

    # ----------------------------------------------------------------
    for child_type_name, link in tqdm.tqdm(child_types.items()):

        # Lay noi dung child type name
        childtype      = ET.SubElement(parenttype, 'childtype')
        childtype.text = child_type_name

        recipe_type_child_tag = ET.SubElement(recipe_type_tag, 'childtype', name=child_type_name)

        # Chuyen toi link khac
        driver.get(link)

        # Tim header p
        recipe_type_child_tag.text = driver.find_element(By.XPATH, '//div[@class="cat-desc col-span-8 px-8 pt-4 md:pt-0 pb-12"]/p').text


FILE_PATH = './data/recipe_types/recipe_types_2.xml'

def prettify_xml(elem):
    rough_string = ET.tostring(elem, encoding="unicode")
    parsed = minidom.parseString(rough_string)
    return parsed.toprettyxml(indent="\t")

with open(FILE_PATH, 'w', encoding='utf-8') as f:
    f.write(prettify_xml(data))

100%|██████████| 8/8 [12:01<00:00, 90.21s/it] 


In [5]:
def save_xml_file(data, file_path: str):
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(prettify_xml(data))

In [ ]:

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)

# actions = ActionChains(driver)

# root cua cac file xml
specified_types = ET.Element('specifiedtype')
count = 0

# -----------------------------------------------------------------
for recipe_type_name, child_types in tqdm.tqdm(recipe_types.items()):

    recipe = ET.Element('recipe')

    # Lay noi dung parent type recipe
    parenttype      = ET.SubElement(recipe, 'parenttype')
    parenttype.text = recipe_type_name

    specified_parenttype = ET.SubElement(specified_types, 'parenttpye', {'name': recipe_type_name})

    # ----------------------------------------------------------------
    for child_type_name, link in tqdm.tqdm(child_types.items()):

        # Lay noi dung child type name
        childtype      = ET.SubElement(parenttype, 'childtype')
        childtype.text = child_type_name

        specified_childtype = ET.SubElement(specified_parenttype, 'childtype', {'name': child_type_name})

        # Chuyen toi link khac
        driver.get(link)

        # Tim cac <section> chua specified type
        sections = driver.find_elements(By.XPATH, '//section[@class="md:max-w-6xl mx-auto px-4 py-12 border-b"]')

        # ------------------------------------------------------------------
        sections_dict = {}
        for section in tqdm.tqdm(sections):

            specified_type_element = ET.SubElement(specified_childtype, 'specifiedtype')
            
            # Lay cac <header> trong section
            section_header = section.find_element(By.TAG_NAME, 'header')
            
            # Lay cac <h2> trong section
            try:
                section_header_h2              = section_header.find_element(By.TAG_NAME, 'h2')
                section_header_h2_element      = ET.SubElement(specified_type_element, 'title')
                section_header_h2_element.text = section_header_h2.text
            except NoSuchElementException as ex:
                print("Exception:", ex.msg)

            # Lay cac <p> trong section header
            try:
                section_header_p              = section_header.find_element(By.TAG_NAME, 'p')
                section_header_p_element      = ET.SubElement(specified_type_element, 'description')
                section_header_p_element.text = section_header_p.text
            except NoSuchElementException as ex:
                print("Exception:", ex.msg)

            # Lay cac ten cua cac mon an co ten loai
            specified_type_recipe_name_element = ET.SubElement(specified_type_element, 'recipes')
            try:
                div_element = section.find_element(By.XPATH, './/div[@class="flickity-viewport"]')
                div_element_soup = BeautifulSoup(div_element.get_attribute("innerHTML"), "html.parser")
                h3_elements = div_element_soup.find_all('h3')
                sections_dict[section_header_h2.text] = set()
                for h3_element in h3_elements:
                    sections_dict[section_header_h2.text].add(h3_element.text)
                    text_tag = ET.SubElement(specified_type_recipe_name_element, 'recipe', {'name': h3_element.text})

            except NoSuchElementException as ex:
                print("Exception:", ex.msg)

        # ------------------------------------------------------------------
        # Xu ly trang dau tien
        articles = driver.find_elements(By.XPATH, '//div[@class="grid grid-cols-12 gap-4"]/article')
        recipe_root = ET.Element('recipe')
        for article in tqdm.tqdm(articles)  :

            parenttype_tag = ET.SubElement(recipe_root, 'parrenttype', {'name': recipe_type_name}) # Them loai parent
            childtype_tag = ET.SubElement(recipe_root, 'childtype', {'name': child_type_name}) # them loai child

            # Lay tat ca anh kich thuoc khac nhau
            recipe_repr_img_urls = article.find_element(By.TAG_NAME, 'img').get_attribute('srcset')
            recipe_repr_img_urls = [recipe_repr_img_url.split()[0] for recipe_repr_img_url in recipe_repr_img_urls.split(", ")]
            img_tag = ET.SubElement(recipe_root, 'images')
            [ET.SubElement(img_tag, 'image', {'url': url}) for url in recipe_repr_img_urls]

            # Lay link
            recipe_link = article.find_element(By.TAG_NAME, 'a')

            # Mo tab moi
            driver.execute_script("window.open(arguments[0].href, '_blank');", recipe_link)
            driver.switch_to.window(driver.window_handles[1])

            # Lay specified type neu co
            article_tag = driver.find_element(By.TAG_NAME, 'article')
            article_header = article_tag.find_element(By.XPATH, './/header/h1')
            for childtype in sections_dict.keys():
                if article_header.text in sections_dict[childtype]:
                    ET.SubElement(recipe_root, 'specifiedtype', {"name": childtype})
                    break
            else:
                ET.SubElement(recipe_root, 'specifiedtype', {"name": ""})

            # Lay ten recipe
            ET.SubElement(recipe_root, 'title', {"name": article_header.text})

            # Lay reviews va sao
            reviews_and_avgstars_tag = article_tag.find_elements(By.XPATH, './/div[@class="text-xxs text-gray-700 tracking-wider md:tracking-extra-widest uppercase font-arvo"]/span')
            ET.SubElement(recipe_root, "reviews", {"quantity": reviews_and_avgstars_tag[0].text.split()[0]}) # Lay so luong reviews
            ET.SubElement(recipe_root, "avgstar", {"average": reviews_and_avgstars_tag[1].text.split()[0]}) # Lay so luong sao trung binh

            # Lay loi trich dan
            header_entry_content = article_tag.find_element(By.XPATH, './/div[@class="mb-6 entry-content italic"]/p')
            header_entry_content_tag = ET.SubElement(recipe_root, 'headdescription')
            header_entry_content_tag.text = header_entry_content.text

            # Lay cau truc cua bai viet
            structure = ET.SubElement(recipe_root, 'structure')
            entry_content = article_tag.find_element(By.XPATH, './/div[@class="entry-content"]')
            content_children = entry_content.find_elements(By.XPATH, './*')
            slice_ = slice(1, len(content_children) - 2)
            for child in content_children[slice_]:
                if child.tag_name == 'figure':
                    ET.SubElement(structure, 'image').text = child.find_element(By.TAG_NAME, 'img').get_attribute('src')
                elif child.tag_name == 'h2' or child.tag_name == 'p':
                    text_tag = ET.SubElement(structure, child.tag_name)
                    text_tag.text = child.text
                elif child.tag_name == 'ul' or child.tag_name == 'ol':
                    list_tag = ET.SubElement(structure, child.tag_name)
                    li_elements = child.find_elements(By.XPATH, './*')
                    for li_element in li_elements:
                        ET.SubElement(list_tag, 'li').text = li_element.text
                elif child.tag_name == 'div':
                    match child.get_attribute("class"):
                        case "featured-comment disable-link-style":
                            ET.SubElement(structure, 'h5').text = child.find_element(By.XPATH, './/div[@class="featured-comment-content"/h5]').text
                            ET.SubElement(structure, 'p').text = child.find_element(By.XPATH, './/div[@class="featured-comment-content"]/h5/div[@class="comment-text w-full"]/p').text
                            break
                        case "adthrive-video-player in-post":
                            ET.SubElement(structure, "video").text = child.find_element(By.XPATH, './/meta[@itemprop="contentUrl"]').get_attribute("content")
                            break
                        case "wp-block-group lindsay-notes":
                            ET.SubElement(structure, "h2").text = child.find_element(By.TAG_NAME, 'h2').text
                            p_tags = child.find_elements(By.TAG_NAME, 'p')
                            p_text = "".join([p_tag.text for p_tag in p_tags])
                            ET.SubElement(structure, 'p').text = p_text
                            break
                        case "schema-faq wp-block-yoast-faq-block":
                            div_tags = child.find_elements(By.XPATH, ".//*")
                            questions = ET.SubElement(structure, "questions")
                            for div_tag in div_tags:
                                ET.SubElement(questions, "strong").text = div_tag.find_element(By.TAG_NAME, "strong").text
                                ET.SubElement(questions, "p").text = div_tag.find_element(By.TAG_NAME, "p").text
                            break
                        case "wp-block-group":
                            div_tags = child.find_elements(By.XPATH, './/div[@class="schema-faq wp-block-yoast-faq-block"]/*')
                            questions = ET.SubElement(structure, "questions")
                            ET.SubElement(questions,'h2').text = child.find_element(By.TAG_NAME, "h2").text
                            for div_tag in div_tags:
                                ET.SubElement(questions, "strong").text = div_tag.find_element(By.TAG_NAME, "strong").text
                                ET.SubElement(questions, "p").text = div_tag.find_element(By.TAG_NAME, "p").text
                            break
                        case "wp-block-group recipe-step":
                            order_id = child.find_element(By.CLASS_NAME, "wp-block-heading large-number")
                            step = ET.SubElement(structure, "step", order=order_id)
                            ET.SubElement(step, "h3").text = child.find_element(By.XPATH, './/h3[@id="h-cook-the-noodles"]').text
                            ET.SubElement(step, 'p').text = child.find_element(By.TAG_NAME, 'p').text
                            [ET.SubElement(step, 'image', url=img_tag.get_attribute('src')) for img_tag in child.find_elements(By.TAG_NAME, 'img')]
                            break
                        case _:
                            ET.SubElement(structure, "div").text = child.text
                            break


            # Lay ngay viet bai
            date = article_tag.find_elements(By.XPATH, './/div[@class="entry-footer"]/div')
            date = datetime.datetime.strptime(date[0].text, "%B %d, %Y")
            ET.SubElement(recipe_root, 'createddate', date=date.date().isoformat())
        
            # Dong tab
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

            # Luu file xml
            save_xml_file(recipe_root, f'./data/recipes/recipe_{count}.xml')

            count += 1

            break

        # Lay so luong trang
        e = driver.find_elements(By.XPATH, '//div[@class="nav-links"]')
        num_pages = int(e[0].find_elements(By.TAG_NAME, 'a')[-2].text) 

        # Xu ly trang thu 2 tro di
        # for i in tqdm.tqdm(range(2, num_pages + 1)):
        #     driver.get(link + f'/page/{i}')
            
        break

    break

# save_xml_file(specified_types, './data/recipe_types/specified_types.xml')  

driver.quit()

In [ ]:
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)
# actions = ActionChains(driver)
driver.get('https://pinchofyum.com/thai-peanut-chicken-bowls')
article = driver.find_element(By.TAG_NAME, 'article')
entry_content = article.find_element(By.XPATH, './/div[@class="entry-content"]')

article.find_element(By.XPATH, './/div[@class="entry-footer"]/')